In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Using dataset sandpit_data which is basically taken from the sandpit series of models, in particular I only used the f_pred_* csv's.
# https://www.kaggle.com/joecooper/tps-oct-joes-sandpit
# The v prefix shows which version of the notebook generated that output.






This notebook is by no means a tutorial on kmean. I'm just trying out a few ideas and hoping that I can learn a little from it. In particular I enjoyed the notebook https://www.kaggle.com/motchan/tps-oct-2021-kmeans
It got me thinking and that is always a good thing. 

I was concerned that the basic idea wouldn't work too well since the 280 features in the training data are just not well correlatated to the tartget. Clustering wouldn't change that but could it help. I was scepticle.
I copied the notebook and ran some tests. I concluded (or at least believed) that it wasn't doing anything helpful. As soon as the sample size increases the value of the cluster features diminihses. Which means that a decent model will find out that info without the need to pre run kmeans. Also the idea of basing the cluster on just a few features didn't feel right. I feel all 280 odd features sit within the same category. Week corrlation with target.

But now I've trained a few models and would hope that they do have a correlation with the target. Can kmeans generate clusters from these predictions. Can it help spot the circumstances that LightGBM is better than Catboost and XGBoost? 

Certainly this concept wasn't included in the list that Raahul had for his experiments in stacking https://www.kaggle.com/raahulsaxena/tps-oct-21-some-experiments-with-stacking/notebook


To set expectations with this notebook - My experience with kmeans would be measured in days (decimal points allowed) rather than years. 
Also - I've not cleaned this notebook - so it largely follows my though patterns and occassionally goes off tangent into deadends.


In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime
import gc

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc

import json
import optuna

import lightgbm as lgbm
import optuna.integration.lightgbm as lgbo
import matplotlib.pyplot as plt

import seaborn as sns
import datatable as dt

from sklearn.metrics import roc_auc_score

from warnings import filterwarnings
filterwarnings('ignore')

from warnings import resetwarnings

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

## Better check what version we are using as I may want to make use of init_model
import lightgbm as lgbm
print (lgbm.__version__)

from sklearn.cluster import KMeans

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
%%time
df_train = reduce_mem_usage(pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv'))
df_test = reduce_mem_usage (dt.fread('../input/tabular-playground-series-oct-2021/test.csv').to_pandas())

sample_submission = pd.read_csv("../input/tabular-playground-series-oct-2021/sample_submission.csv")

In [ ]:
# del df_kmeans_train
df_kmeans_train = pd.DataFrame(np.zeros(df_train.shape[0])).copy()
df_kmeans_test = pd.DataFrame(np.zeros(df_test.shape[0])).copy()

In [ ]:

df_kmeans_train['v6_lgbm1'] = pd.read_csv("../input/sandpit-data/v6_f_pred_train.csv")['pred']
df_kmeans_train['v7_xgb0'] = pd.read_csv("../input/sandpit-data/v7_f_pred_train.csv")['pred']
df_kmeans_train['v8_xgb1'] = pd.read_csv("../input/sandpit-data/v8_f_pred_train.csv")['pred']
df_kmeans_train['v10_catb0'] = pd.read_csv("../input/sandpit-data/v10_f_pred_train.csv")['pred']
df_kmeans_train['v11_lgbm0'] = pd.read_csv("../input/sandpit-data/v11_f_pred_train.csv")['pred']


df_kmeans_test['v6_lgbm1'] = pd.read_csv("../input/sandpit-data/v6_f_pred_test.csv")['pred']
df_kmeans_test['v7_xgb0'] = pd.read_csv("../input/sandpit-data/v7_f_pred_test.csv")['pred']
df_kmeans_test['v8_xgb1'] = pd.read_csv("../input/sandpit-data/v8_f_pred_test.csv")['pred']
df_kmeans_test['v10_catb0'] = pd.read_csv("../input/sandpit-data/v10_f_pred_test.csv")['pred']
df_kmeans_test['v11_lgbm0'] = pd.read_csv("../input/sandpit-data/v11_f_pred_test.csv")['pred']


joe_features = ['v6_lgbm1', 'v7_xgb0', 'v8_xgb1', 'v10_catb0', 'v11_lgbm0']

In [ ]:
y = df_train['target']

In [ ]:
from sklearn.metrics import roc_auc_score

for col in joe_features:
    score = roc_auc_score(y, df_kmeans_train[col])
    print ("Joes overall computed score for ", col, " is ", score)

In [ ]:
# Joes overall computed score for  v10_catb0  is  0.8566436011399643   0.85597
# Joes overall computed score for  v8_xgb1    is  0.8568621253215738   0.85621
# Joes overall computed score for  v11_lgbm0  is  0.8569640619014858   0.85638
# Joes overall computed score for  v7_xgb0    is  0.8570048237818386   0.85645
# Joes overall computed score for  v6_lgbm1   is  0.8571127311293687   0.85633

# How well does that correlation between my CV and LB score fit

## Not fantastic, but it is what it is




In [ ]:
def diff_checker (col):

    print ("Working on : ", col)

    diffs_df = df_kmeans_train.copy()
    diffs_df ['diff'] = (diffs_df[col]-y).abs()

    print ("Correct prediction")
    print (diffs_df['diff'].between(0, 0.5, inclusive=False).value_counts())
    print ()
    print ()
    
    
# diff_checker ('v6_lgbm1')

In [ ]:
for col in joe_features:
    diff_checker (col)

In [ ]:
## Stealing some code from https://www.kaggle.com/motchan/tps-oct-2021-kmeans


useful_features = joe_features
distortions=[]
for i in range(1,14):
    print ("woring on : ", i)
    km = KMeans(n_clusters=i,
              init="k-means++",
              max_iter=500,
              random_state=42)
    km.fit(df_kmeans_train[useful_features])
    distortions.append(km.inertia_)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1,14),distortions,marker="o")
plt.xticks(range(1,14))
plt.xlabel("Number of clusters")
plt.ylabel("Distortion")
plt.show()

In [ ]:
# ## Excatly what is this graph supposed to be telling me ?





In [ ]:
feature_cols = [col for col in df_kmeans_test.columns.tolist()]

In [ ]:
%%time
# useful_features = ["f22","f179","f69","f156","f58","f136","f214"]

## okay lets drop the number of cluster to 6 
n_clusters_1 = 6
cd_feature = True # cluster distance instead of cluster number
cluster_cols = [f"cluster{i+1}" for i in range(n_clusters_1)]
kmeans = KMeans(n_clusters=n_clusters_1, init="k-means++", max_iter=500, random_state=42)

if cd_feature:
    # train
    X_cd = kmeans.fit_transform(df_kmeans_train[useful_features])
    X_cd = pd.DataFrame(X_cd, columns=cluster_cols, index=df_kmeans_train.index)
    df_kmeans_train = df_kmeans_train.join(X_cd)
    # test
    X_cd = kmeans.transform(df_kmeans_test[useful_features])
    X_cd = pd.DataFrame(X_cd, columns=cluster_cols, index=df_kmeans_test.index)
    df_kmeans_test = df_kmeans_test.join(X_cd)
    
else:
    print ("don't bother")
#     # train
#     train["cluster"] = kmeans.fit_predict(train[useful_features])
#     # test
#     test["cluster"] = kmeans.predict(test[useful_features])
    
#     # one-hot encode
#     ohe = OneHotEncoder()
#     X_ohe = ohe.fit_transform(np.array(train["cluster"]).reshape(-1,1)).toarray()
#     T_ohe = ohe.transform(np.array(test["cluster"]).reshape(-1,1)).toarray()

#     X_ohe = pd.DataFrame(X_ohe, columns=cluster_cols, index=train.index)
#     T_ohe = pd.DataFrame(T_ohe, columns=cluster_cols, index=test.index)

#     train = pd.concat([train, X_ohe],axis=1)
#     test = pd.concat([test, T_ohe],axis=1)

feature_cols += cluster_cols
df_kmeans_train.head()

In [ ]:
# feature_cols = [col for col in df_kmeans_test.columns.tolist()]
# feature_cols += cluster_cols
df_kmeans_train.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
fig = plt.figure(figsize = (20,5))

if cd_feature:
    sns.kdeplot(data=df_kmeans_train[cluster_cols])
else:
    ax = sns.countplot(data=df_kmeans_train, x='cluster', hue="target")
    for p in ax.patches:
        ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.2, p.get_height()), ha='center', va='top', color='white', size=5)

plt.show()

In [ ]:
# okay - that IS a cool graph. I can see some patterns, but still don't know what it is showing me. 
# To be more precise : I'm not clear on exactly how it is supossed to help me move forward or what alterations would need to be made to improve things. 

In [ ]:
df_kmeans_train['target'] = df_train['target']

In [ ]:
%%time
from sklearn.feature_selection import mutual_info_regression
### how dangerous is it to sample on only 5000 records
x = df_kmeans_train.iloc[:5000,:][feature_cols].copy()
y = df_kmeans_train.iloc[:5000,:]['target'].copy()
mi_scores = mutual_info_regression(x, y)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=x.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [ ]:
import plotly.figure_factory as ff
import plotly.express as px
top = 20
fig = px.bar(mi_scores, x=mi_scores.values[:top], y=mi_scores.index[:top])
fig.update_layout(
    title=f"Top {top} Strong Relationships Between Feature Columns and Target Column",
    xaxis_title="Relationship with Target",
    yaxis_title="Feature Columns",
    yaxis={'categoryorder':'total ascending'},
    colorway=["blue"]
)
fig.show()

In [ ]:
%%time
from sklearn.feature_selection import mutual_info_regression

### how dangerous is it to sample on only 5000 records?
### Try with a value of 50K failed with ram issues

x = df_kmeans_train.iloc[:30000,:][feature_cols].copy()
y = df_kmeans_train.iloc[:30000,:]['target'].copy()
mi_scores = mutual_info_regression(x, y)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=x.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [ ]:
import plotly.figure_factory as ff
import plotly.express as px
top = 20
fig = px.bar(mi_scores, x=mi_scores.values[:top], y=mi_scores.index[:top])
fig.update_layout(
    title=f"Top {top} Strong Relationships Between Feature Columns and Target Column",
    xaxis_title="Relationship with Target",
    yaxis_title="Feature Columns",
    yaxis={'categoryorder':'total ascending'},
    colorway=["blue"]
)
fig.show()

In [ ]:
# So as I increase the sample size (from 5k to 10k to 15k to 40k - 50k gives ram issues) the importance of clusters drops and instead we have my base models becoming more important. 
# At least that gives me some hope that my base models have some correlation with the traget and I haven't been wasting time over the last few weeks.
# However I'm thinking that maybe my own CV scores or even the LB scores are a much simpler way of working that all out.



In [ ]:
# Anyway my DF's are now populated with some new features that I don't understand very well. At least not the ones that match "cluster+i"
# So I should use this opportunity to do some modelling against them and see what the output looks like

print (df_kmeans_train.columns)
print (df_kmeans_test.columns)



In [ ]:
X = df_kmeans_train.drop(columns=[ "target"]).copy()
y = df_kmeans_train["target"].copy()

test_data = df_kmeans_test.copy()

In [ ]:
xgb_params = {
   'eval_metric': 'auc',     'n_estimators': 1000,  'learning_rate' : 0.01       }

# I have no idea what params would be good for this dataset. 

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

oof_pred_tmp = dict()
test_pred_tmp = dict()
scores_tmp = dict()

name = 'defXGB'

oof_pred_tmp[name] = list()
oof_pred_tmp["y_valid"] = list()
test_pred_tmp[name] = list()
scores_tmp[name] = list()


### I need a basic model
## nomally logisticRegression is a good choice, but the addition of kmeans might change that. 
model = XGBClassifier(**xgb_params)

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    print ("Fold : ", fold, " Start : ", datetime.now())
    start_time = datetime.now()
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid,y_valid)],
        early_stopping_rounds=80,
        verbose=100
    )

    pred_valid = model.predict_proba(X_valid)[:, -1]
    score = roc_auc_score(y_valid, pred_valid)

    scores_tmp[name].append(score)
    oof_pred_tmp[name].extend(pred_valid)

    end_time = datetime.now()
    duration = end_time - start_time

#     f_pred.loc[idx_valid, 'pred'] += model.predict_proba(X_valid)[:, 1]/len(SEEDS)
#     f_test.loc[f_test.index, 'pred'] += model.predict_proba(test_data)[:, 1]/len(SEEDS)/N_FOLDS  ## 5 being number of folds

    print(f"Fold: {fold} Model: {name} Score: {score}", "Duration :", duration)

    ### Time to get the predictions for competition test data (df_test descendandts)
    y_hat = model.predict_proba(test_data)[:,1]
    test_pred_tmp[name].append(y_hat)

    oof_pred_tmp["y_valid"].extend(y_valid)


print(f"Average Validation Score | {name}: {np.mean(scores_tmp[name])}")

In [ ]:
# Time to unravel those dictionaries of models and predictions 
test_predictions = pd.DataFrame(  
    {name: np.mean(np.column_stack(test_pred_tmp[name]), axis=1) for name in test_pred_tmp.keys()}
)

# Always handy to save a copy for use later
test_predictions.to_csv("./test_predictions.csv", index=False)

# get the average of all predictions
test_predictions["avg"] = test_predictions.mean(axis=1)

# overwrite target in sample_submission with our new and improved predictions 
avg_submission = sample_submission.copy()
avg_submission["target"] = test_predictions["avg"]
avg_submission.to_csv("./avg_submission1.csv", index=False)

In [ ]:
# and before we even think about submitting - we should check what out oof cv looks like


oof_predictions = pd.DataFrame({name:oof_pred_tmp[name] for name in oof_pred_tmp.keys()})

# Again Always handy so save a copy for revisiting
oof_predictions.to_csv("./oof_preds.csv", index=False)

# And now we get the all important validation score
y_valid = oof_predictions["y_valid"].copy()
y_hat_blend = oof_predictions.drop(columns=["y_valid"]).mean(axis=1)
score = roc_auc_score(y_valid, y_hat_blend)

print(f"Overall Training Validation Score | Blend: {score}")


In [ ]:
### At this stage we don't have a BAD CV score. Is it worth running the last few sections again after dropping the cluster features. We can compare the results and see if Kmeans clustering had any influence.

In [ ]:
cluster_features =  ['cluster1',  'cluster2',  'cluster3',  'cluster4', 'cluster5',  'cluster6']

In [ ]:
for clusterf in cluster_features:
    df_kmeans_train = df_kmeans_train.drop(clusterf, 1)
    df_kmeans_test = df_kmeans_test.drop(clusterf, 1)
    

In [ ]:
X = df_kmeans_train.drop(columns=[ "target"]).copy()
y = df_kmeans_train["target"].copy()
test_data = df_kmeans_test.copy()

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

oof_pred_tmp = dict()
test_pred_tmp = dict()
scores_tmp = dict()

name = 'defXGB'

oof_pred_tmp[name] = list()
oof_pred_tmp["y_valid"] = list()
test_pred_tmp[name] = list()
scores_tmp[name] = list()


### I need a basic model
model = XGBClassifier(**xgb_params)

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    print ("Fold : ", fold, " Start : ", datetime.now())
    start_time = datetime.now()
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid,y_valid)],
        early_stopping_rounds=80,
        verbose=100
    )

    pred_valid = model.predict_proba(X_valid)[:, -1]
    score = roc_auc_score(y_valid, pred_valid)

    scores_tmp[name].append(score)
    oof_pred_tmp[name].extend(pred_valid)

    end_time = datetime.now()
    duration = end_time - start_time

#     f_pred.loc[idx_valid, 'pred'] += model.predict_proba(X_valid)[:, 1]/len(SEEDS)
#     f_test.loc[f_test.index, 'pred'] += model.predict_proba(test_data)[:, 1]/len(SEEDS)/N_FOLDS  ## 5 being number of folds

    print(f"Fold: {fold} Model: {name} Score: {score}", "Duration :", duration)

    ### Time to get the predictions for competition test data (df_test descendandts)
    y_hat = model.predict_proba(test_data)[:,1]
    test_pred_tmp[name].append(y_hat)

    oof_pred_tmp["y_valid"].extend(y_valid)


print(f"Average Validation Score | {name}: {np.mean(scores_tmp[name])}")

In [ ]:
test_predictions = pd.DataFrame(      {name: np.mean(np.column_stack(test_pred_tmp[name]), axis=1) for name in test_pred_tmp.keys()})

test_predictions["avg"] = test_predictions.mean(axis=1)

avg_submission = sample_submission.copy()
avg_submission["target"] = test_predictions["avg"]
avg_submission.to_csv("./avg_submission2.csv", index=False)

In [ ]:
# and before we even think about submitting - we should check what out oof cv looks like


oof_predictions = pd.DataFrame({name:oof_pred_tmp[name] for name in oof_pred_tmp.keys()})

# Again Always handy so save a copy for revisiting
oof_predictions.to_csv("./oof_preds.csv", index=False)

# And now we get the all important validation score
y_valid = oof_predictions["y_valid"].copy()
y_hat_blend = oof_predictions.drop(columns=["y_valid"]).mean(axis=1)
score = roc_auc_score(y_valid, y_hat_blend)

print(f"Overall Training Validation Score | Blend: {score}")


In [ ]:

## Initail run suggests a difference 
## Maybe tuning the final model params instead would make a difference
## Or running a differnt model.  XGBoost was not that good an idea?
## Maybe tuning the kmeans model might yield improvments as clearly I didn't understand what it was doing and choosing 6 clusters was maybe not a good idea.


In [ ]:
%%time

# A last quick test of the logisticRegression code from original sandpit series
## https://www.kaggle.com/joecooper/tps-oct-joes-sandpit

from sklearn.linear_model import LogisticRegression

# get the set of predictions for oof data (training descendants)
X_meta = oof_predictions.drop(columns=["y_valid"]).copy()
y_meta = oof_predictions["y_valid"].copy()
test_meta = test_predictions.drop(columns=["avg"]).copy()


### Notice how we can simplify this when only working with one model
### we no longer need to create (and then later unravel) dictionaries
meta_pred_tmp = []
scores_tmp = []

## This modelling has so few features that it costs nothing to increase folds
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

for fold, (idx_train, idx_valid) in enumerate(kf.split(X_meta, y_meta)):
    X_train, y_train = X_meta.iloc[idx_train], y_meta.iloc[idx_train]
    X_valid, y_valid = X_meta.iloc[idx_valid], y_meta.iloc[idx_valid]

    ### Obviously params could be added to tune the model, training is quick but 
    ## benefits might not be significant. The effort shoudl be expended before here 
    ## such that you have better data to work with
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # validation prediction
    pred_valid = model.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, pred_valid)
    scores_tmp.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    
    y_hat = model.predict_proba(test_meta)[:,1]
    meta_pred_tmp.append(y_hat)
    

print(f"Meta Validation Score | Meta: {np.mean(scores_tmp)}")

In [ ]:
# For perspective we started with
# Joes overall computed score for  v10_catb0  is  0.8566436011399643   0.85597
# Joes overall computed score for  v8_xgb1    is  0.8568621253215738   0.85621
# Joes overall computed score for  v11_lgbm0  is  0.8569640619014858   0.85638
# Joes overall computed score for  v7_xgb0    is  0.8570048237818386   0.85645
# Joes overall computed score for  v6_lgbm1   is  0.8571127311293687   0.85633


In [ ]:
# get the average prediction for each fold of meta data
meta_preds = np.mean(np.column_stack(meta_pred_tmp), axis=1)

# create submission file
meta_submission = sample_submission.copy()
meta_submission["target"] = meta_preds
meta_submission.to_csv("./meta_submission.csv", index=False)